In [1]:
import os
import sys
from gensim.models.ldamodel import LdaModel
import gensim
import pandas as pd
import glob
from nltk.corpus import stopwords
import string
from gensim import corpora
import timeit
from collections import defaultdict
from nltk import PorterStemmer
from gensim.utils import lemmatize # Only keep nums
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

C:\Users\mnan\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\mnan\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
print(sys.version_info) ### Make sure you are using Python3

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


In [3]:
## change dir
os.chdir(os.getcwd()+'\data')

dict_data = {}
country_list = []
content_list = []        
for i in glob.glob('*.txt'):
    with open(i,encoding='utf-8',errors = 'ignore') as f:
        country_list += [i.split('.')[0]]
        content = f.readlines()
        content_list += [''.join(list(filter(None, list(map(lambda x:x.strip(), content)))))]
dict_data['country'] = country_list
dict_data['content'] = content_list
df_all = pd.DataFrame(dict_data)

In [4]:
cd ../

C:\Users\mnan\Desktop\un-project


In [5]:
## change dir
os.chdir(os.getcwd()+'\profiles')

dictionary_data = {}
dict_country_list = []
dict_content_list = []        
for i in glob.glob('*.txt'):
    with open(i,encoding='utf-8',errors = 'ignore') as f:
        dict_country_list += [i.split('.')[0]]
        dict_content = f.readlines()
        dict_content_list += [''.join(list(filter(None, list(map(lambda x:x.strip(), dict_content)))))]
dictionary_data['country'] = dict_country_list
dictionary_data['content'] = dict_content_list
dict_df_all = pd.DataFrame(dictionary_data)

In [6]:
df_all.head()

,content,country
0,IslamicRepublicofAfghanistanMinistryofCommunic...,Afghanistan
1,Shkresa-KokaColorR E P U B L I K A E S H Q I P...,Albania (nonEnglish)
2,AUSTRALIASCYBER SECURITY STRATEGYEnabling inno...,Australia
3,Logo Republik sterreichLogo Bundeskanzleramt s...,Austria
4,Az.rbaycan Prezidentinin R.smi internet s.hif....,Azerbaijan (nonEnglish)


In [7]:
dict_df_all.head()

,content,country
0,﻿----------------------- Page 1---------------...,Afghanistan
1,﻿----------------------- Page 1---------------...,Albania
2,﻿----------------------- Page 1---------------...,Algeria
3,﻿----------------------- Page 1---------------...,Andorra
4,﻿----------------------- Page 1---------------...,Angola


In [8]:
def process_text(paragraph_list):
    """
    stopwords
    
    punctuation
    
    digits
    
    
    """
    typical_words = ['security','cyber','the','information','government'
                     ,'cybersecurity','national','development','systems'
                     ,'services','strategy','international','cyberspace']
    stoplist = stopwords.words('english') + typical_words 
    
    remove_digits = str.maketrans('', '', string.digits)
    remove_punctuation = str.maketrans('', '', string.punctuation)
    
    
    lemma = WordNetLemmatizer()
    
    
    paragraph_cleaned =[i.translate(remove_digits) for i in paragraph_list]
    
    paragraph_cleaned = [i.translate(remove_punctuation) for i in paragraph_cleaned]
    
    paragraph_cleaned = [[lemma.lemmatize(word.lower()) for word in line.split() 
                          if (word.lower() not in stoplist and len(word)>4)] for line in paragraph_cleaned]

    return paragraph_cleaned

In [9]:
paragraph_cleaned = process_text(content_list)

In [10]:
dict_paragraph_cleaned = process_text(dict_content_list)

## bigram model

In [11]:
bigram = gensim.models.Phrases(dict_paragraph_cleaned,min_count=1, threshold=2)

In [12]:
dictionary_text_list = list(bigram[dict_paragraph_cleaned])

C:\Users\mnan\Anaconda3\lib\site-packages\gensim\models\phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [13]:
train_text_list = list(bigram[paragraph_cleaned])

C:\Users\mnan\Anaconda3\lib\site-packages\gensim\models\phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## dictionary built

In [14]:
dictionary = Dictionary(dictionary_text_list)
corpus = [dictionary.doc2bow(text) for text in train_text_list]

In [15]:
pwd

'C:\\Users\\mnan\\Desktop\\un-project\\profiles'

In [16]:
cd ../

C:\Users\mnan\Desktop\un-project


## lda modeling

In [36]:
topic_num = 5
lda = LdaModel(corpus, num_topics= topic_num,id2word = dictionary)

## Dump 

In [37]:
with open('lda.pickle', 'wb') as handle:
    pickle.dump(lda, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('corpus.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)